In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
db = client.tripadvisor
collection = db.reviews

In [14]:
url = 'https://www.tripadvisor.com/Restaurant_Review-g28970-d622172-Reviews-Blue_Duck_Tavern-Washington_DC_District_of_Columbia.html'
browser.visit(url)

In [ ]:
# session = requests.Session()
# session.headers.update({'User-Agent': 'Custom user agent'})
# html = session.get(url)

In [15]:
for p in range(1,20):
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    revs = soup.find_all('div', class_='review-container')
    
    for r in revs:
        
        id = r.find('div', class_='reviewSelector')['data-reviewid']
        title = r.find('span', class_='noQuotes').text
        date = r.find('span', class_='ratingDate relativeDate')['title']
        name = r.find('div', class_='username mo').text
        rating = r.find('div', class_='rating reviewItemInline').find('span')['class'][1][7:8]
        text = r.find('div', class_='entry').text
         
        try:
            loc = r.find('span', class_='expand_inline userLocation')
            if loc.text:
                location = loc.text
        except AttributeError:
            print('no location')

        review = {
            'id': id,
            'date': date,
            'title': title,
            'rating': rating,
            'text': text,
            'username': name
        }
    
        if (location):
            review['userlocation']= location
        else:
            review['userlocation']='no_location'
    
    
        print(f'page: {p}')
        print(review)
        print('--------------------')
        
        collection.insert_one(review)
    
        review = {}
        title = ''
        date = ''
        name = ''
        rating = ''
        text = ''
        location = ''
    
    browser.click_link_by_text('Next')   

page: 1
{'id': '579961251', 'date': 'May 13, 2018', 'title': "Fantastic Mother's Day Experience", 'rating': '5', 'text': "We ate here for Mother's day and received Michelin star customer service. Adrienne was very welcoming and had great attention to detail. The food was absolutely amazing, especially the bone marrow dish. Theresa Mascariñas", 'username': 'ewattsie', 'userlocation': 'London, United Kingdom'}
--------------------
page: 1
{'id': '579287644', 'date': 'May 10, 2018', 'title': 'Great Meal, Worthy of a Michelin Star', 'rating': '5', 'text': 'We returned for dinner at the Blue Duck Tavern and were far more impressed than on previous occasions. The foie gros creme brulee was a unique and delicious appetizer. The rack of lamb (enough for two persons) was exceptional, although quite expensive, and the whole...More', 'username': 'cluelessSingapore', 'userlocation': 'Fairfax, Virginia'}
--------------------
no location
page: 1
{'id': '578678261', 'date': 'May 7, 2018', 'title': 'N

In [ ]:
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
revs = soup.find_all('div', class_='review-container')

In [ ]:
for r in revs:
    
    id = r.find('div', class_='reviewSelector')['data-reviewid']
    title = r.find('span', class_='noQuotes').text
    date = r.find('span', class_='ratingDate relativeDate')['title']
    name = r.find('div', class_='username mo').text
    rating = r.find('div', class_='rating reviewItemInline').find('span')['class'][1][7:8]
    text = r.find('div', class_='entry').text
    
    try:
        loc = r.find('span', class_='expand_inline userLocation')
        if loc.text:
            location = loc.text
    except AttributeError:
        print('no location')
    
    review = {
        'id': id,
        'date': date,
        'title': title,
        'rating': rating,
        'text': text,
        'username': name
    }
    
    if (location):
        review['userlocation']= location
    else:
        review['userlocation']='no_location'
    
    
    
    print(review)
    print('--------------------')
    
#     collection.insert_one(review)
    
    review = {}
    title = ''
    date = ''
    name = ''
    rating = ''
    text = ''
    location = ''

In [ ]:
data = db.reviews.find()
for rev in data:
    print(rev)

In [ ]:
try:
    for revs[3].find('div', class_='entry'):
        browser.click_link_by_partial_text('next')
except ElementDoesNotExist:
    print("Scraping Complete")